In [5]:
!pip install git+https://github.com/charliewolf/pynder.git
!pip install opencv-python
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip list

  Cloning https://github.com/charliewolf/pynder.git to /tmp/pip-req-build-wrp63616
  Running setup.py bdist_wheel for pynder ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-glhdoovi/wheels/fc/7c/83/d287f87f5f3ad1f44307649403e94a31d78eb38304f7f128de
Successfully built pynder
  Using cached https://files.pythonhosted.org/packages/45/bd/e0a4391ac105ecf73a6e14372174b05774634c7c6454e49c38750d516eee/opencv_python-4.0.0.21-cp37-cp37m-manylinux1_x86_64.whl
Package                            Version   
---------------------------------- ----------
alabaster                          0.7.12    
anaconda-client                    1.7.2     
anaconda-navigator                 1.9.6     
anaconda-project                   0.8.2     
asn1crypto                         0.24.0    
astroid                            2.1.0     
astropy                            3.1       
atomicwrites                       1.2.1     
attrs                              18.2.0    
Babel                         

In [7]:
!pip install httplib2

    100% |████████████████████████████████| 225kB 2.2MB/s ta 0:00:011
  Running setup.py bdist_wheel for httplib2 ... done
  Stored in directory: /home/riku/.cache/pip/wheels/98/82/0d/cfb126a5e40d487157e43fdb3332937713dde36b4d0e1e754d
Successfully built httplib2


In [9]:
import os
import re
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np
import urllib.request
from urllib.parse import quote
import httplib2
import pynder
import pandas as pd
import time

In [37]:
def get_image(img_url): # ウェブ上の画像をダウンロードする
    opener = urllib.request.build_opener()
    http = httplib2.Http(".cache")
    response, content = http.request(img_url)            
    return content

def aidemy_imshow(name, img): # Jupyter notebook上で画像を表示
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    plt.imshow(img)
    plt.show()
cv2.imshow = aidemy_imshow

def jpg_count(folder_name): # ディレクトリ内のjpgファイルを数える
    files = os.listdir("./"+folder_name)
    jpgcount = 0
    for file in files:
        index = re.search(".jpg", file)
        if index:
            jpgcount += 1
    return jpgcount

def image_save(img,folder_name): # 画像を保存する
    # ディレクトリがなければ作る
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    jpgcount = jpg_count(folder_name)
    # カウント数+1でファイル名をつけて保存
    w_pass = "./{}/{}.jpg".format(folder_name,jpgcount)
    cv2.imwrite(w_pass,img)
    
def image_save_time(img,folder_name,time): # timestampをつけて画像を保存する
    # ディレクトリがなければ作る
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    # timestampをつけて保存
    w_pass = "./{}/{}.jpg".format(folder_name,time)
    cv2.imwrite(w_pass,img)
    
def unique_url_judge(df,url):#新しい画像か判定
    df_bool = (df['photoURL']==url)
    return df_bool.sum()

In [40]:
usrid = "Facebook ID" # findmyfbidで取得したFacebook ID
token = "Token" # Facebook Tinder Token

session = pynder.Session(facebook_id = usrid, facebook_token = token)

In [48]:
# coding: UTF-8

# 写真を保存するディレクトリを作成する
dir_name = "image_tin" 
if not os.path.exists(dir_name):
        os.mkdir(dir_name)

session = pynder.Session(token)
#friends = session.get_fb_friends()

# 探索する都市の緯度と経度を指定

# Locations = [["Sapporo", 43.055248, 141.345505],
#              ["Akita", 40.824589, 140.755203],
#              ["Utunomiya", 36.554241, 139.897705],
#              ["Tokyo", 35.680909, 139.767372],
#              ["Yokohama", 35.465786, 139.622313],
#              ["Niigata", 37.917793, 139.06105],
#              ["Nagoya", 35.154919, 136.920593],
#              ["Kyoto", 35.009129, 135.754807],
#              ["Osaka", 34.702509, 135.496505],
#              ["Hukuoka", 33.579788, 130.402405],
#              ["Okinawa", 26.204830, 127.692398]
#             ]

Locations = [["Akita", 40.824589, 140.755203],
            ]

# 各都市で取得する人数の上限
Limit = 3000

cols = ["location","data_id","name","age", "photoURL", "photoURL2", "instagramID","distance", "school","job","time"]
df = pd.DataFrame(index=[], columns=cols)

for location in Locations:
    print(location[0],location[1],location[2])
    session.update_location(location[1], location[2])
    users = session.nearby_users()
    df2 = pd.DataFrame(index=[], columns=cols)    
    count = 0
    for user in users:
        count += 1
        if count > Limit :
            break
        photo_urls = user.get_photos(width="640")
        username = user.name 
        userage = user.age
        #userbd = user.birth_date
        #userthumbnail = user.thumbnail[0]
        userphoto = list(user.photos)[0]
        userphoto2 = list(user.get_photos(width="640"))[0]
        userinstagram = user.instagram_username
        userdistance = user.distance_km # distane from you
        userschool = user.schools
        userjob = user.jobs
        now = int(time.time()*100)
        
        if unique_url_judge(df,userphoto)==0:
        
            #userinstaphoto = [user.instagram_photos][0]
            print(username, userage, userdistance, userphoto)
            se = pd.Series([location[0],count-1, username, userage,userphoto, userphoto2,userinstagram, userdistance, userschool, userjob,now],index=df.columns)    
            df = df.append(se, ignore_index=True)
            df2 = df.append(se, ignore_index=True)
            img_buf = np.fromstring(get_image(userphoto2), dtype='uint8')
            img = cv2.imdecode(img_buf, 1)
            image_save_time(img,"./" + dir_name + "/"+"{}".format(location[0]),str(now))
        else:
            print("This image isn't unique.")
            
    print(location[0] + " is done")

    print("Finished Scraping. Writing CSV.......")
    df2.to_csv("./" + dir_name + "/"+"{}".format(location[0])+"_output.csv")

print("Finished Scraping. Writing CSV.......")
df.to_csv("./" + dir_name + "/"+"totaloutput.csv")    
print("DONE")

RequestError: Couldn't authenticate

In [44]:
df_header = pd.read_csv('/home/riku/tin/image_tin/Sapporo_output.csv')


In [45]:
print(df_header['photoURL'].nunique())

1136


In [38]:
unique_url_judge(df_header,userphoto)

2